# 参数初始化
参数初始化对模型具有较大的影响，不同的初始化方式可能会导致截然不同的结果，所幸的是很多深度学习的先驱们已经帮我们探索了各种各样的初始化方式，所以我们只需要学会如何对模型的参数进行初始化的赋值即可。

PyTorch 的初始化方式并没有那么显然，如果你使用最原始的方式创建模型，那么你需要定义模型中的所有参数，当然这样你可以非常方便地定义每个变量的初始化方式，但是对于复杂的模型，这并不容易，而且我们推崇使用 Sequential 和 Module 来定义模型，所以这个时候我们就需要知道如何自定义初始化方式

## 使用 NumPy 来初始化
因为 PyTorch 是一个非常灵活的框架，理论上能够对所有的 Tensor 进行操作，所以我们能够通过定义新的 Tensor 来初始化，直接看下面的例子

In [1]:
import numpy as np
import torch
from torch import nn

In [2]:
# 定义一个 Sequential 模型
net1 = nn.Sequential(
    nn.Linear(30, 40),
    nn.ReLU(),
    nn.Linear(40, 50),
    nn.ReLU(),
    nn.Linear(50, 10)
)

In [3]:
# 访问第一层的参数
w1 = net1[0].weight
b1 = net1[0].bias

In [4]:
print(w1)

Parameter containing:
-0.1239  0.0942 -0.1171  ...  -0.1558  0.0137 -0.0923
-0.0387 -0.1354 -0.0754  ...   0.0857 -0.1287 -0.1284
 0.1626 -0.0725 -0.0277  ...  -0.0894  0.0082 -0.1295
          ...             ⋱             ...          
 0.0728  0.0827 -0.1092  ...   0.1563 -0.0889 -0.1495
 0.0945  0.0926 -0.1045  ...  -0.0625 -0.0216  0.1414
-0.1409  0.0349  0.1544  ...   0.1582  0.0114 -0.0134
[torch.FloatTensor of size 40x30]



注意，这是一个 Parameter，也就是一个特殊的 Variable，我们可以访问其 `.data`属性得到其中的数据，然后直接定义一个新的 Tensor 对其进行替换，我们可以使用 PyTorch 中的一些随机数据生成的方式，比如 `torch.randn`，如果要使用更多 PyTorch 中没有的随机化方式，可以使用 numpy

In [5]:
# 定义一个 Tensor 直接对其进行替换
net1[0].weight.data = torch.from_numpy(np.random.uniform(3, 5, size=(40, 30)))

In [6]:
print(net1[0].weight)

Parameter containing:
 3.7358  3.5705  4.0640  ...   4.8674  4.2839  3.1047
 3.2760  3.9492  4.3779  ...   4.6799  3.8670  4.2164
 4.6651  4.8062  3.4228  ...   3.8087  3.9535  4.1640
          ...             ⋱             ...          
 3.5799  3.4089  3.8735  ...   4.7839  3.0517  3.1685
 4.2341  3.5010  3.4424  ...   3.3558  3.4131  4.6690
 3.4293  4.1870  3.0100  ...   4.5652  4.5963  3.8731
[torch.DoubleTensor of size 40x30]



可以看到这个参数的值已经被改变了，也就是说已经被定义成了我们需要的初始化方式，如果模型中某一层需要我们手动去修改，那么我们可以直接用这种方式去访问，但是更多的时候是模型中相同类型的层都需要初始化成相同的方式，这个时候一种更高效的方式是使用循环去访问，比如

In [7]:
for layer in net1:
    if isinstance(layer, nn.Linear): # 判断是否是线性层
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 
        # 定义为均值为 0，方差为 0.5 的正态分布

**小练习：一种非常流行的初始化方式叫 Xavier，方法来源于 2010 年的一篇论文 [Understanding the difficulty of training deep feedforward neural networks](http://proceedings.mlr.press/v9/glorot10a.html)，其通过数学的推到，证明了这种初始化方式可以使得每一层的输出方差是尽可能相等的，有兴趣的同学可以去看看论文**

我们给出这种初始化的公式

$$
w\ \sim \ Uniform[- \frac{\sqrt{6}}{\sqrt{n_j + n_{j+1}}}, \frac{\sqrt{6}}{\sqrt{n_j + n_{j+1}}}]
$$

其中 $n_j$ 和 $n_{j+1}$ 表示该层的输入和输出数目，所以请尝试实现以下这种初始化方式

对于 Module 的参数初始化，其实也非常简单，如果想对其中的某层进行初始化，可以直接像 Sequential 一样对其 Tensor 进行重新定义，其唯一不同的地方在于，如果要用循环的方式访问，需要介绍两个属性，children 和 modules，下面我们举例来说明

In [8]:
class sim_net(nn.Module):
    def __init__(self):
        super(sim_net, self).__init__()
        self.l1 = nn.Sequential(
            nn.Linear(30, 40),
            nn.ReLU()
        )
        
        self.l1[0].weight.data = torch.randn(40, 30) # 直接对某一层初始化
        
        self.l2 = nn.Sequential(
            nn.Linear(40, 50),
            nn.ReLU()
        )
        
        self.l3 = nn.Sequential(
            nn.Linear(50, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.l1(x)
        x =self.l2(x)
        x = self.l3(x)
        return x

In [9]:
net2 = sim_net()

In [10]:
# 访问 children
for i in net2.children():
    print(i)

Sequential(
  (0): Linear(in_features=30, out_features=40)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=40, out_features=50)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=50, out_features=10)
  (1): ReLU()
)


In [11]:
# 访问 modules
for i in net2.modules():
    print(i)

sim_net(
  (l1): Sequential(
    (0): Linear(in_features=30, out_features=40)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=40, out_features=50)
    (1): ReLU()
  )
  (l3): Sequential(
    (0): Linear(in_features=50, out_features=10)
    (1): ReLU()
  )
)
Sequential(
  (0): Linear(in_features=30, out_features=40)
  (1): ReLU()
)
Linear(in_features=30, out_features=40)
ReLU()
Sequential(
  (0): Linear(in_features=40, out_features=50)
  (1): ReLU()
)
Linear(in_features=40, out_features=50)
ReLU()
Sequential(
  (0): Linear(in_features=50, out_features=10)
  (1): ReLU()
)
Linear(in_features=50, out_features=10)
ReLU()


通过上面的例子，看到区别了吗?

children 只会访问到模型定义中的第一层，因为上面的模型中定义了三个 Sequential，所以只会访问到三个 Sequential，而 modules 会访问到最后的结构，比如上面的例子，modules 不仅访问到了 Sequential，也访问到了 Sequential 里面，这就对我们做初始化非常方便，比如

In [12]:
for layer in net2.modules():
    if isinstance(layer, nn.Linear):
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 

这上面实现了和 Sequential 相同的初始化，同样非常简便

## torch.nn.init
因为 PyTorch 灵活的特性，我们可以直接对 Tensor 进行操作从而初始化，PyTorch 也提供了初始化的函数帮助我们快速初始化，就是 `torch.nn.init`，其操作层面仍然在 Tensor 上，下面我们举例说明

In [13]:
from torch.nn import init

In [14]:
print(net1[0].weight)

Parameter containing:
 0.4151 -0.5196 -0.0961  ...   0.2016 -0.9218  0.2034
-0.7896  0.0754 -0.3235  ...  -0.4686 -0.5014  0.6814
 0.1516  0.1913 -0.3432  ...   0.3969 -0.3046  0.3304
          ...             ⋱             ...          
-0.5187  0.5876  0.0740  ...  -0.4041  0.3274  0.2361
-0.3881  0.4708 -0.1102  ...  -0.0504 -1.0549  0.6737
-0.4326 -0.3389 -0.2350  ...  -0.2523 -0.3381 -0.6509
[torch.DoubleTensor of size 40x30]



In [15]:
# 这就是上面我们讲过的 Xavier 初始化方法，PyTorch 直接内置了其实现
init.xavier_uniform(net1[0].weight) 

Parameter containing:
 0.1573  0.1963 -0.1683  ...  -0.1839 -0.2883 -0.2148
-0.1422  0.2575 -0.0471  ...  -0.1098  0.1848 -0.2919
-0.1048  0.2204 -0.2238  ...  -0.1491  0.2507 -0.2878
          ...             ⋱             ...          
 0.2844 -0.0312 -0.0502  ...  -0.2301 -0.1848 -0.0894
 0.0875 -0.2216 -0.0637  ...   0.1879  0.0350 -0.2889
-0.2331  0.1786  0.0590  ...   0.0317  0.1908  0.0314
[torch.DoubleTensor of size 40x30]

In [16]:
print(net1[0].weight)

Parameter containing:
 0.1573  0.1963 -0.1683  ...  -0.1839 -0.2883 -0.2148
-0.1422  0.2575 -0.0471  ...  -0.1098  0.1848 -0.2919
-0.1048  0.2204 -0.2238  ...  -0.1491  0.2507 -0.2878
          ...             ⋱             ...          
 0.2844 -0.0312 -0.0502  ...  -0.2301 -0.1848 -0.0894
 0.0875 -0.2216 -0.0637  ...   0.1879  0.0350 -0.2889
-0.2331  0.1786  0.0590  ...   0.0317  0.1908  0.0314
[torch.DoubleTensor of size 40x30]



可以看到参数已经被修改了

`torch.nn.init` 为我们提供了更多的内置初始化方式，避免了我们重复去实现一些相同的操作

上面讲了两种初始化方式，其实它们的本质都是一样的，就是去修改某一层参数的实际值，而 `torch.nn.init` 提供了更多成熟的深度学习相关的初始化方式，非常方便

下一节课，我们将讲一下目前流行的各种基于梯度的优化算法